In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

teams = pd.read_csv("../data/march_madness_2024.csv")

In [ ]:
from march_madness.tournament import Tournament

tournament = Tournament(teams=teams)

unplayed_games = tournament.get_unplayed_games()
print(f"Round 1: {len(unplayed_games)} games")
for game in unplayed_games:
    tournament.update_game_result(game, game.team1)

unplayed_games = tournament.get_unplayed_games()
print(f"Round 2: {len(unplayed_games)} games")
for game in unplayed_games:
    tournament.update_game_result(game, game.team1)
    
unplayed_games = tournament.get_unplayed_games()
print(f"Round 3: {len(unplayed_games)} games")
for game in unplayed_games:
    tournament.update_game_result(game, game.team1)
    
unplayed_games = tournament.get_unplayed_games()
print(f"Round 4: {len(unplayed_games)} games")
for game in unplayed_games:
    tournament.update_game_result(game, game.team1)

unplayed_games = tournament.get_unplayed_games()
print(f"Round 5: {len(unplayed_games)} games")
for game in unplayed_games:
    tournament.update_game_result(game, game.team1)
    
unplayed_games = tournament.get_unplayed_games()
print(f"Round 6: {len(unplayed_games)} games")
for game in unplayed_games:
    tournament.update_game_result(game, game.team1)
    

In [ ]:
from march_madness.prediction import random_prediction, team1_always_wins
from march_madness.tournament import Tournament, TournamentSimulator

simulator = TournamentSimulator(num_trials=10,
                                tournament_class=Tournament,
                                tournament_params={"teams": teams},
                                prediction_strategy=random_prediction,
                                seed=42)

simulator.run(verbose=True)
df = simulator.to_dataframe()
df.head()

In [ ]:
from march_madness.prediction import llm_prediction
from march_madness.tournament import Tournament, TournamentSimulator

simulator = TournamentSimulator(num_trials=100,
                                tournament_class=Tournament,
                                tournament_params={"teams": teams},
                                prediction_strategy=llm_prediction,
                                prediction_strategy_kwargs={
                                    "model_name": "llama3.2:1b"
                                },
                                result_path="../results/llama3_2-1b.csv",
                                seed=42)

simulator.run(resume=True, verbose=True)
df = simulator.to_dataframe()
df.head()

In [ ]:
df["game_id_str"] = df["game_id"].astype(str)

trial_brackets = df.groupby("trial")[["game_id_str", "game_winner"]].apply(lambda x: tuple(zip(x["game_id_str"], x["game_winner"])))

bracket_counts = trial_brackets.value_counts().reset_index()
bracket_counts.columns = ["bracket", "count"]

total_trials = df["trial"].nunique()
bracket_counts["percentage"] = (bracket_counts["count"] / total_trials) * 100

# most common full bracket
best_bracket_tuple = bracket_counts.iloc[0]["bracket"]  # Most frequent bracket
best_bracket_df = pd.DataFrame(best_bracket_tuple, columns=["game_id_str", "game_winner"])

# add frequency and percentage for each game
game_frequencies = df.groupby(["game_id_str", "game_winner"]).size().reset_index(name="game_count")
game_frequencies["game_percentage"] = (game_frequencies["game_count"] / total_trials) * 100

best_bracket_df = best_bracket_df.merge(game_frequencies, on=["game_id_str", "game_winner"], how="left")

best_bracket_df = best_bracket_df.drop(columns=["game_id_str"])

best_bracket_df

# weight later rounds heavier?